In [ ]:
!pip install pandas matplotlib seaborn pytesseract opencv-python deepface tweepy instaloader transformers beautifulsoup4 requests pillow
!sudo apt install tesseract-ocr

In [ ]:
!sudo apt install tesseract-ocr-por
import os
import pytesseract

os.environ['TESSDATA_PREFIX'] = '/usr/share/tesseract-ocr/4.00/tessdata'

In [ ]:
# %% [markdown]
"""
## Cadastro de Fãs
"""
# %%
import pandas as pd
from ipywidgets import widgets, Layout
from IPython.display import display, clear_output
import re

cor_principal = '#000000'
cor_secundaria = '#FFFFFF'
cor_destaque = '#FFD700'

# Estilos para feedback e botões
style_valid = {'description_style': 'success'}
style_invalid = {'description_style': 'danger'}
style_botao = {'button_color': cor_destaque, 'font_weight': 'bold', 'color': cor_principal}

# Estilo para o título
titulo_style = f"color: {cor_destaque}; font-size: 2em; font-weight: bold;"
label_color = cor_principal

# DataFrame para armazenar os dados
df_fans = pd.DataFrame(columns=[
    'Nome', 'CPF', 'Email', 'Jogo_Favorito_Principal',
    'Outros_Jogos', 'Interesses_Conteudo', 'Plataformas_Conteudo',
    'Frequencia_Acompanhamento', 'Participacao_Comunidades', 'Idade'
])

# Widgets para coleta de dados
nome = widgets.Text(
    description=f"<span style='color: {label_color};'>Nome:*</span>",
    layout=Layout(width='500px'),
    style=style_invalid,
    tooltip="Digite seu nome completo."
)
cpf = widgets.Text(
    description=f"<span style='color: {label_color};'>CPF:</span>",
    layout=Layout(width='500px'),
    tooltip="Digite seu CPF no formato XXX.XXX.XXX-XX."
)
email = widgets.Text(
    description=f"<span style='color: {label_color};'>Email:*</span>",
    layout=Layout(width='500px'),
    style=style_invalid,
    tooltip="Digite seu email no formato correto (ex: usuario@dominio.com)."
)
jogo_principal = widgets.Dropdown(
    options=['CS2', 'Valorant', 'League of Legends', 'DOTA 2', 'Outro'],
    description=f"<span style='color: {label_color};'>Jogo Favorito Principal:*</span>",
    style={'description_width': 'initial'}
)
outros_jogos = widgets.Text(
    description=f"<span style='color: {label_color};'>Outros Jogos (separados por vírgula):</span>",
    layout=Layout(width='500px'),
    tooltip="Especifique outros jogos que você acompanha.",
    style={'description_width': 'initial'}
)
interesses_conteudo = widgets.SelectMultiple(
    options=['Streams', 'Vídeos de Highlights', 'Notícias', 'Campeonatos', 'Outros'],
    description=f"<span style='color: {label_color};'>Interesses em Conteúdo:</span>",
    layout=Layout(height='100px'),
    style={'description_width': 'initial'}
)
plataformas_conteudo = widgets.SelectMultiple(
    options=['Twitch', 'YouTube', 'Instagram', 'Twitter', 'Outras'],
    description=f"<span style='color: {label_color};'>Plataformas de Conteúdo:</span>",
    layout=Layout(height='100px'),
    style={'description_width': 'initial'}
)
frequencia = widgets.Dropdown(
    options=['Diariamente', 'Semanalmente', 'Mensalmente', 'Raramente'],
    description=f"<span style='color: {label_color};'>Frequência de Acompanhamento:</span>",
    style={'description_width': 'initial'}
)
participacao_comunidades = widgets.Dropdown(
    options=['Sim', 'Não'],
    description=f"<span style='color: {label_color};'>Participa de Comunidades Online?</span>",
    style={'description_width': 'initial'}
)
idade = widgets.IntSlider(
    description=f"<span style='color: {label_color};'>Idade (opcional):</span>",
    min=0,
    max=120,
    step=1,
    value=18,
    continuous_update=False,
    layout=Layout(width='500px'),
    style={'description_width': 'initial'}
)
btn_salvar = widgets.Button(description="Salvar Dados", style=style_botao)
output = widgets.Output()

# Funções de validação de inserção de dados
def is_valid_email(email_str):
    return re.match(r"[^@]+@[^@]+\.[^@]+", email_str)

def is_valid_cpf(cpf_str):
    cpf_str = ''.join(filter(str.isdigit, cpf_str))
    if len(cpf_str) != 11:
        return False
    if cpf_str == cpf_str[0] * 11:
        return False
    return True

# Função para salvar os dados
def salvar_dados(b):
    with output:
        clear_output()
        dados_validos = True
        if not nome.value:
            nome.style = style_invalid
            print("O campo Nome é obrigatório.")
            dados_validos = False
        else:
            nome.style = style_valid

        if not email.value or not is_valid_email(email.value):
            email.style = style_invalid
            print("Por favor, insira um Email válido.")
            dados_validos = False
        else:
            email.style = style_valid

        if not jogo_principal.value:
            print("O campo Jogo Favorito Principal é obrigatório.")
            dados_validos = False

        cpf_valido = True
        if cpf.value and not is_valid_cpf(cpf.value):
            cpf.style = style_invalid
            print("Por favor, insira um CPF válido.")
            cpf_valido = False
        elif cpf.value:
            cpf.style = style_valid
        else:
            cpf.style = {}

        if dados_validos:
            global df_fans
            new_data = {
                'Nome': nome.value,
                'CPF': cpf.value if cpf_valido else None,
                'Email': email.value,
                'Jogo_Favorito_Principal': jogo_principal.value,
                'Outros_Jogos': [jogo.strip() for jogo in outros_jogos.value.split(',')] if outros_jogos.value else [],
                'Interesses_Conteudo': list(interesses_conteudo.value),
                'Plataformas_Conteudo': list(plataformas_conteudo.value),
                'Frequencia_Acompanhamento': frequencia.value,
                'Participacao_Comunidades': participacao_comunidades.value,
                'Idade': idade.value if idade.value > 0 else None
            }
            df_fans = pd.concat([df_fans, pd.DataFrame([new_data])], ignore_index=True)
            clear_output()
            display(widgets.HTML(f"<h3 style='color: {cor_destaque};'>Dados salvos com sucesso!</h3>"))
            display(df_fans.tail())

btn_salvar.on_click(salvar_dados)

display(widgets.HTML(f"<h2 style='{titulo_style}'>Cadastro de Fãs</h2>"))
display(nome, cpf, email, jogo_principal, outros_jogos,
        interesses_conteudo, plataformas_conteudo, frequencia,
        participacao_comunidades, idade, btn_salvar, output)

In [ ]:
# %% [markdown]
!pip install --upgrade deepface opencv-python tensorflow
"""
## Upload e Validação de Documentos com IA
"""
# %%
from IPython.display import display, HTML, Image
import os
import cv2
import pytesseract
from deepface import DeepFace
from PIL import Image as PILImage
import numpy as np
import io
import ipywidgets as widgets
import base64

# Configuração do Tesseract OCR
os.environ['TESSDATA_PREFIX'] = '/usr/share/tesseract-ocr/4.00/tessdata'
pytesseract.pytesseract.tesseract_cmd = '/usr/bin/tesseract'

# Widget para upload
upload_doc = widgets.FileUpload(description="Documento de Identificação", accept='.jpg,.png,.jpeg', multiple=False)
upload_selfie = widgets.FileUpload(description="Selfie", accept='.jpg,.png,.jpeg', multiple=False)
btn_validar = widgets.Button(description="Validar Documentos")

def image_to_base64(image_path):
    """Converte imagem para base64 para exibição no notebook"""
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode('utf-8')

def pil_to_base64(pil_img):
    """Converte imagem PIL para base64"""
    buffered = io.BytesIO()
    pil_img.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

# Função para processar documentos
def validar_documentos(b):
    if not upload_doc.value or not upload_selfie.value:
        print("Por favor, envie ambos os arquivos.")
        return

    try:
        # Processa documento
        doc_content = next(iter(upload_doc.value.values()))
        doc_img = PILImage.open(io.BytesIO(doc_content['content']))
        if doc_img.mode == 'RGBA':
            doc_img = doc_img.convert('RGB')

        # Processa selfie
        selfie_content = next(iter(upload_selfie.value.values()))
        selfie_img = PILImage.open(io.BytesIO(selfie_content['content']))
        if selfie_img.mode == 'RGBA':
            selfie_img = selfie_img.convert('RGB')

        # Salva temporariamente para OCR
        doc_img.save('temp_doc.jpg')
        selfie_img.save('temp_selfie.jpg')

        # Converte para base64 para exibição
        doc_base64 = pil_to_base64(doc_img)
        selfie_base64 = pil_to_base64(selfie_img)

        # Validação do documento com OCR
        img = cv2.imread('temp_doc.jpg')
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        text = pytesseract.image_to_string(gray, lang='por')

        # Verificação facial com tratamento de erro
        try:
            result = DeepFace.verify(
                img1_path='temp_doc.jpg',
                img2_path='temp_selfie.jpg',
                model_name='VGG-Face',
                detector_backend='opencv'
            )
            facial_result = f"{result['verified']} (Distância: {result['distance']:.2f})"
        except Exception as e:
            facial_result = f"Erro na verificação: {str(e)}"

        # Verificação de documento válido
        doc_valido = any(word in text.upper() for word in ['CPF', 'RG', 'IDENTIDADE', 'BRASIL'])

        # Exibe resultados com imagens
        display(HTML(f"""
        <div style="margin: 20px 0;">
            <h3>Resultado da Validação</h3>
            <div style="display: flex; justify-content: space-around; margin: 20px 0;">
                <div style="text-align: center;">
                    <h4>Documento</h4>
                    <img src="data:image/jpeg;base64,{doc_base64}" style="max-width: 300px; border: 2px solid #ddd; border-radius: 5px;">
                </div>
                <div style="text-align: center;">
                    <h4>Selfie</h4>
                    <img src="data:image/jpeg;base64,{selfie_base64}" style="max-width: 300px; border: 2px solid #ddd; border-radius: 5px;">
                </div>
            </div>
            <div style="background: #f5f5f5; padding: 15px; border-radius: 5px; color: #333;">
                <p><strong>Texto extraído:</strong> {text[:200]}...</p>
                <p><strong>Verificação facial:</strong> {facial_result}</p>
                <p><strong>Documento válido:</strong> {doc_valido}</p>
            </div>
        </div>
        """))

    except Exception as e:
        print(f"Erro na verificação: {str(e)}")
    finally:
        # Limpa arquivos temporários
        if os.path.exists('temp_doc.jpg'):
            os.remove('temp_doc.jpg')
        if os.path.exists('temp_selfie.jpg'):
            os.remove('temp_selfie.jpg')

btn_validar.on_click(validar_documentos)

# Exibindo a seção
display(HTML("<h2 style='color: #1a5276;'>Validação de Documentos</h2>"))
display(widgets.HBox([upload_doc, upload_selfie]))
display(btn_validar)

In [ ]:
# %% [markdown]
"""
##  FURIA Fan Analytics Dashboard
### Análise de Perfil para Scouting e Marketing
"""
# %%
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from ipywidgets import Layout
import random
import time
import re

FURIA_THEME = {
    'primary': '#FFD700',   # Dourado
    'secondary': '#000000',   # Preto
    'accent': '#FF4500'     # Laranja
}

PLATFORM_ICONS = {
    'instagram': '📷',
    'twitter': '🐦',
    'twitch': '🎮'
}

# %% [markdown]
###  Módulo de Análise FURIA
# %%
class FuriaAnalyzer:
    def __init__(self):
        self.furia_players = ['FalleN', 'mwzera', 'gabssf']

    def analyze_instagram(self, username):
        time.sleep(0.5)
        furia_engagement = random.randint(5, 50)
        return {
            'username': username,
            'metrics': {
                'followers': random.randint(1000, 10000),
                'furia_engagement': furia_engagement,
                'engagement_rate': f"{random.uniform(3, 8):.1f}%",
                'esports_posts': random.randint(5, 50),
                'furia_mentions': random.randint(0, furia_engagement)
            },
            'content': {
                'top_hashtags': ['#furia', '#csgo', '#gaming', '#esports', random.choice(['#cs2moments', '#KSCERATO', '#FURIAFC'])],
                'player_interactions': {player: random.randint(0, 20) for player in self.furia_players}
            }
        }

    def analyze_twitter(self, username):
        time.sleep(0.5)
        furia_interactions = random.randint(5, 100)
        return {
            'username': username,
            'metrics': {
                'followers': random.randint(500, 5000),
                'furia_interactions': furia_interactions,
                'tweet_frequency': f"{random.randint(1, 10)}/dia",
                'esports_mentions': random.randint(5, 100),
                'furia_retweets': random.randint(0, furia_interactions//2)
            },
            'interactions': {
                'top_players': random.sample(self.furia_players, 3),
                'furia_hashtags': ['#FURIALoL ', '#FURIACS', random.choice(['#FURIAPUBG', '#FURIAwin'])]
            }
        }

    def analyze_twitch(self, username):
        time.sleep(0.5)
        furia_streams = random.randint(0, 10)
        return {
            'username': username,
            'metrics': {
                'followers': random.randint(100, 5000),
                'avg_viewers': random.randint(10, 500),
                'hours_watched': random.randint(10, 1000),
                'furia_streams_watched': furia_streams,
                'furia_cheers': random.randint(0, furia_streams*5)
            },
            'preferences': {
                'games': ['CS2', 'Valorant', 'League of Legends'],
                'furia_players_watched': random.sample(self.furia_players, random.randint(0, 3)),
                'furia_events': random.choice(['ESL Impact', 'CBLOL Academy', 'PGL Astana', None])
            }
        }

    def calculate_fan_score(self, data):
        """Calcula o FURIA Fan Score (0-100)"""
        platform = 'twitter' if 'tweet_frequency' in data['metrics'] else 'instagram' if 'engagement_rate' in data['metrics'] else 'twitch'

        if platform == 'instagram':
            base = data['metrics']['furia_mentions'] * 2
            base += data['metrics']['furia_engagement']
            base += sum(data['content']['player_interactions'].values())
        elif platform == 'twitter':
            base = data['metrics']['furia_retweets'] * 3
            base += data['metrics']['furia_interactions']
        else:   # twitch
            base = data['metrics']['furia_streams_watched'] * 4
            base += data['metrics']['furia_cheers']
            base += len(data['preferences']['furia_players_watched']) * 10

        # Normaliza para 0-100
        score = min(100, base)
        data['furia_fan_score'] = score
        data['fan_tier'] = self.get_fan_tier(score)
        return data

    def get_fan_tier(self, score):
        """Classifica o fã em categorias"""
        if score >= 80: return "Super Fã 🐆🔥"
        if score >= 50: return "Fã Engajado ⚡"
        if score >= 20: return "Fã Casual 👍"
        return "Observador 👀"

# %% [markdown]
### Iniciar o Dashboard FURIA
# %%
class FuriaDashboard:
    def __init__(self):
        self.analyzer = FuriaAnalyzer()
        self.setup_ui()

    def setup_ui(self):
        """Configura a interface do dashboard"""
        style = """
        <style>
            .furia-header {
                background: linear-gradient(90deg, #000000 0%, #FFD700 100%);
                color: white;
                padding: 15px;
                border-radius: 8px;
                text-align: center;
                margin-bottom: 20px;
            }
            .furia-card {
                border-left: 4px solid #FFD700;
                padding: 10px 15px;
                margin: 10px 0;
                background: #f8f8f8;
                border-radius: 0 8px 8px 0;
            }
        </style>
        """
        display(HTML(style))

        # Widgets de entrada
        self.instagram_input = widgets.Text(
            placeholder='instagram.com/username',
            description='Instagram:',
            layout=Layout(width='90%')
        )

        self.twitter_input = widgets.Text(
            placeholder='twitter.com/username',
            description='Twitter:',
            layout=Layout(width='90%')
        )

        self.twitch_input = widgets.Text(
            placeholder='twitch.tv/username',
            description='Twitch:',
            layout=Layout(width='90%')
        )

        self.analyze_btn = widgets.Button(
            description='Analisar Fã',
            button_style='warning',
            icon='search',
            layout=Layout(width='200px')
        )
        self.analyze_btn.on_click(self.analyze_profiles)

        self.output = widgets.Output()

        # Layout
        display(HTML(
            "<div class='furia-header'>"
            "<h1>FURIA Fan Analytics</h1>"
            "<p>Identifique e analise fãs da FURIA nas redes sociais</p>"
            "</div>"
        ))

        display(widgets.VBox([
            self.instagram_input,
            self.twitter_input,
            self.twitch_input,
            self.analyze_btn,
            self.output
        ]))

    def analyze_profiles(self, b):
        """Processa as redes sociais inseridas"""
        with self.output:
            clear_output()
            display(HTML("<div style='text-align:center;'><h3>🔍 Analisando perfis...</h3></div>"))

            results = {}
            platforms = {
                'instagram': self.instagram_input.value,
                'twitter': self.twitter_input.value,
                'twitch': self.twitch_input.value
            }

            for platform, url in platforms.items():
                if url:
                    username = self.extract_username(url, platform)
                    if username:
                        data = getattr(self.analyzer, f'analyze_{platform}')(username)
                        data = self.analyzer.calculate_fan_score(data)
                        results[platform] = data

            if results:
                self.display_results(results)
            else:
                display(HTML("<p style='color:red;text-align:center;'>Nenhuma URL válida foi fornecida</p>"))

    def extract_username(self, url, platform):
        try:
            patterns = {
                'instagram': r'(?:https?:\/\/)?(?:www\.)?instagram\.com\/([a-zA-Z0-9_.]+)/?$',
                'twitter': r'(?:https?:\/\/)?(?:www\.)?twitter\.com\/([a-zA-Z0-9_]+)/?$',
                'twitch': r'(?:https?:\/\/)?(?:www\.)?twitch\.tv\/([a-zA-Z0-9_]+)/?$'
            }
            match = re.search(patterns[platform], url.strip(), re.IGNORECASE)
            return match.group(1).lower() if match else None
        except:
            return None

    def display_results(self, results):
        display(HTML(
            "<div style='text-align:center; margin-bottom:20px;'>"
            f"<h2 style='color:{FURIA_THEME['primary']};'>Relatório de Engajamento com a FURIA</h2>"
            "</div>"
        ))

        # 1. Resumo do Fã
        self.display_fan_summary(results)

        # 2. Análise por plataforma
        for platform in ['instagram', 'twitter', 'twitch']:
            if platform in results:
                self.display_platform_analysis(platform, results[platform])

    def display_fan_summary(self, results):
        """Mostra o resumo consolidado do fã"""
        # Calcula scores médios
        total_score = sum(data['furia_fan_score'] for data in results.values())
        avg_score = total_score / len(results) if results else 0
        overall_tier = self.analyzer.get_fan_tier(avg_score)

        # Cria cards para cada plataforma
        platform_cards = []
        for platform, data in results.items():
            platform_cards.append(
                f"<div style='display:inline-block; width:30%; margin:5px; vertical-align:top;'>"
                f"<div class='furia-card'>"
                f"<h4>{PLATFORM_ICONS[platform]} {platform.capitalize()}</h4>"
                f"<p>Score: <strong style='color:{FURIA_THEME['primary']};'>{data['furia_fan_score']}/100</strong></p>"
                f"<p>Tipo: {data['fan_tier']}</p>"
                f"</div>"
                f"</div>"
            )

        display(HTML(
            f"<div class='furia-card'>"
            f"<h3>📊 Resumo do Fã</h3>"
            f"<p style='font-size:1.2em;'>Score Médio: <strong style='color:{FURIA_THEME['primary']}; font-size:1.3em;'>{avg_score:.1f}/100</strong></p>"
            f"<p style='font-size:1.1em;'>Classificação: <strong>{overall_tier}</strong></p>"
            f"<div style='margin-top:15px;'>" + ''.join(platform_cards) + "</div>"
            f"</div>"
        ))

    def display_platform_analysis(self, platform, data):
        """Mostra análise detalhada por plataforma de forma compacta"""
        color = FURIA_THEME['primary']

        display(HTML(
            f"<div class='furia-card'>"
            f"<h3>{PLATFORM_ICONS[platform]} {platform.capitalize()} - @{data['username']}</h3>"
            f"<p><strong>Score:</strong> <span style='color:{color};'>{data['furia_fan_score']}/100</span></p>"
            f"<p><strong>Nível:</strong> {data['fan_tier']}</p>"
            f"</div>"
        ))

        # Métricas de engajamento com a FURIA
        display(HTML(f"<h4><span style='color:{color};'> </span> [ Engajamento com a FURIA ]</h4>"))
        metrics_html = "<div style='display: grid; grid-template-columns: repeat(auto-fill, minmax(150px, 1fr)); gap: 10px; margin-bottom: 15px;'>"
        for metric, value in data['metrics'].items():
            if 'furia' in metric:
                metrics_html += (
                    f"<div style='background: #fff; padding: 8px; border-radius: 5px; border: 1px solid {color}; text-align: center;'>"
                    f"<strong style='font-size: 0.9em;'>{metric.replace('_', ' ').title()}</strong><br>"
                    f"<span style='color:{color}; font-size: 1.0em;'>{value}</span>"
                    f"</div>"
                )
        metrics_html += "</div>"
        display(HTML(metrics_html))

        # Interações relevantes (listas)
        analysis_key = 'content' if platform == 'instagram' else 'interactions' if platform == 'twitter' else 'preferences'
        if analysis_key in data:
            analysis = data[analysis_key]
            for section, items in analysis.items():
                if isinstance(items, list) and ('furia' in section.lower() or any('furia' in str(item).lower() for item in items)):
                    display(HTML(f"<p><strong>{section.replace('_', ' ').title()}:</strong> {', '.join(map(str, items))}</p>"))
                elif isinstance(items, dict) and ('furia' in section.lower() or any('furia' in str(key).lower() for key in items.keys())):
                    display(HTML(f"<p><strong>{section.replace('_', ' ').title()}:</strong></p>"))
                    if section == 'player_interactions':
                        fig = px.bar(
                            x=list(items.keys()),
                            y=list(items.values()),
                            color=list(items.keys()),
                            color_discrete_sequence=[color]*len(items),
                            labels={'x': 'Jogador', 'y': 'Interações'},
                            title='Interações com Jogadores da FURIA'
                        )
                        fig.show()
                    else:
                        display(HTML("<ul>" + "".join(f"<li style='font-size: 0.9em;'>{k}: {v}</li>" for k, v in items.items()) + "</ul>"))

# %% [markdown]
### Iniciar o Dashboard FURIA
# %%
furia_dashboard = FuriaDashboard()

In [ ]:
# %% [markdown]
"""
##  Dashboard de Engajamento com o Ecossistema FURIA
### Análise de Perfil para Marketing e Comunidade
"""
# %%
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from ipywidgets import Layout
import random
import time
import re

FURIA_THEME = {
    'primary': '#FFD700',   # Dourado
    'secondary': '#000000',   # Preto
    'accent': '#FF4500'     # Laranja
}

PLATFORM_ICONS = {
    'instagram': '📷',
    'twitter': '🐦',
    'twitch': '🎮'
}

# %% [markdown]
###  Módulo de Análise do Ecossistema FURIA
# %%
class FuriaEcoAnalyzer:
    def __init__(self):
        self.furia_teams = ['CS:GO', 'Valorant', 'R6', 'PUBG', 'LoL']
        self.furia_players = ['GUIGO', 'KSCERATO', 'yuurih', 'FalleN', 'Tutsz']
        self.furia_streamers = ['gaules', 'mount', 'brino']
        self.furia_events = []
        self.furia_sponsors = []
        self.most_watched_genres = ['FPS', 'MOBA', 'Battle Royale', 'Estratégia', 'Simulação']

    def analyze_instagram(self, username):
        """Análise simulada de engajamento com FURIA no Instagram"""
        time.sleep(0.5)
        furia_engagement_score = random.randint(10, 90)
        followed_players = random.sample(self.furia_players, random.randint(0, len(self.furia_players)))
        hashtag_mentions = [f'#FURIA{team.replace(":", "")}' for team in random.sample(self.furia_teams, random.randint(0, len(self.furia_teams)))] + ['#DIADEFURIA', '#csclips']

        return {
            'username': username,
            'platform': 'instagram',
            'metrics': {
                'followers': random.randint(1000, 10000),
                'engagement_rate': f"{random.uniform(2, 7):.1f}%",
                'esports_related_posts': random.randint(5, 40)
            },
            'furia_engagement': {
                'followed_players': followed_players,
                'hashtag_mentions': hashtag_mentions
            },
            'furia_engagement_score': furia_engagement_score
        }

    def analyze_twitter(self, username):
        """Análise simulada de engajamento com FURIA no Twitter"""
        time.sleep(0.5)
        furia_engagement_score = random.randint(15, 95)
        player_mentions = random.sample(self.furia_players, random.randint(0, len(self.furia_players)))
        team_mentions = [f'FURIA {team}' for team in random.sample(self.furia_teams, random.randint(0, len(self.furia_teams)))]

        return {
            'username': username,
            'platform': 'twitter',
            'metrics': {
                'followers': random.randint(500, 5000),
                'tweet_frequency': f"{random.randint(1, 8)}/dia",
                'esports_related_tweets': random.randint(10, 80)
            },
            'furia_engagement': {
                'player_mentions': player_mentions,
                'team_mentions': team_mentions
            },
            'furia_engagement_score': furia_engagement_score
        }

    def analyze_twitch(self, username):
        """Análise simulada de engajamento com FURIA no Twitch"""
        time.sleep(0.5)
        furia_engagement_score = random.randint(20, 100)
        watched_streamers = random.sample(self.furia_streamers, random.randint(0, len(self.furia_streamers)))
        chat_mentions = random.randint(0, 20) # Menções de termos FURIA no chat
        most_watched = random.sample(self.most_watched_genres, random.randint(0, len(self.most_watched_genres)))

        return {
            'username': username,
            'platform': 'twitch',
            'metrics': {
                'followers': random.randint(100, 5000),
                'avg_viewers': random.randint(10, 300),
                'hours_watched': random.randint(20, 1500)
            },
            'furia_engagement': {
                'watched_streamers': watched_streamers,
                'chat_mentions': chat_mentions,
                'most_watched_genres': most_watched
            },
            'furia_engagement_score': furia_engagement_score
        }

    def calculate_furia_score(self, data):
        """Calcula o Score de Engajamento com o Ecossistema FURIA (0-100)"""
        base_score = data.get('furia_engagement_score', 50)
        data['overall_furia_score'] = min(100, max(0, base_score))
        data['furia_fan_tier'] = self.get_furia_fan_tier(data['overall_furia_score'])
        return data

    def get_furia_fan_tier(self, score):
        """Classifica o fã por nível de engajamento com a FURIA"""
        if score >= 80: return "Super Fã 🐆🔥"
        if score >= 50: return "Fã Engajado ⚡"
        if score >= 20: return "Apoiador 👍"
        return "Observador 👀"

# %% [markdown]
### Dashboard de Engajamento FURIA
# %%
class FuriaEcoDashboard:
    def __init__(self):
        self.analyzer = FuriaEcoAnalyzer()
        self.setup_ui()

    def setup_ui(self):
        """Configura a interface do dashboard"""
        style = """
        <style>
            .furia-header {
                background: linear-gradient(90deg, #000000 0%, #FFD700 100%);
                color: white;
                padding: 15px;
                border-radius: 8px;
                text-align: center;
                margin-bottom: 20px;
            }
            .furia-card {
                border-left: 4px solid #FFD700;
                padding: 10px 15px;
                margin: 10px 0;
                background: #f8f8f8;
                border-radius: 0 8px 8px 0;
            }
        </style>
        """
        display(HTML(style))

        # Widgets de entrada
        self.instagram_input = widgets.Text(placeholder='instagram.com/username', description='Instagram:', layout=Layout(width='90%'))
        self.twitter_input = widgets.Text(placeholder='twitter.com/username', description='Twitter:', layout=Layout(width='90%'))
        self.twitch_input = widgets.Text(placeholder='twitch.tv/username', description='Twitch:', layout=Layout(width='90%'))
        self.analyze_btn = widgets.Button(description='Analisar Engajamento FURIA', button_style='warning', icon='search', layout=Layout(width='250px'))
        self.analyze_btn.on_click(self.analyze_profiles)
        self.output = widgets.Output()

        # Layout
        display(HTML(
            "<div class='furia-header'>"
            "<h1>Análise de Engajamento com o Ecossistema FURIA</h1>"
            "<p>Descubra o quanto os fãs interagem com a FURIA nas redes sociais</p>"
            "</div>"
        ))

        display(widgets.VBox([
            self.instagram_input,
            self.twitter_input,
            self.twitch_input,
            self.analyze_btn,
            self.output
        ]))

    def analyze_profiles(self, b):
        """Processa as redes sociais inseridas"""
        with self.output:
            clear_output()
            display(HTML("<div style='text-align:center;'><h3>🔍 Analisando perfis...</h3></div>"))

            results = {}
            platforms = {
                'instagram': self.instagram_input.value,
                'twitter': self.twitter_input.value,
                'twitch': self.twitch_input.value
            }

            for platform, url in platforms.items():
                if url:
                    username = self.extract_username(url, platform)
                    if username:
                        data = getattr(self.analyzer, f'analyze_{platform}')(username)
                        data = self.analyzer.calculate_furia_score(data)
                        results[platform] = data

            if results:
                self.display_results(results)
            else:
                display(HTML("<p style='color:red;text-align:center;'>Nenhuma URL válida foi fornecida</p>"))

    def extract_username(self, url, platform):
        """Extrai username da URL"""
        try:
            patterns = {
                'instagram': r'(?:https?:\/\/)?(?:www\.)?instagram\.com\/([a-zA-Z0-9_.]+)/?$',
                'twitter': r'(?:https?:\/\/)?(?:www\.)?twitter\.com\/([a-zA-Z0-9_]+)/?$',
                'twitch': r'(?:https?:\/\/)?(?:www\.)?twitch\.tv\/([a-zA-Z0-9_]+)/?$'
            }
            match = re.search(patterns[platform], url.strip(), re.IGNORECASE)
            return match.group(1).lower() if match else None
        except:
            return None

    def display_results(self, results):
        """Exibe os resultados de engajamento com a FURIA"""
        display(HTML(
            "<div style='text-align:center; margin-bottom:20px;'>"
            f"<h2 style='color:{FURIA_THEME['primary']};'>Relatório de Engajamento com o Ecossistema FURIA</h2>"
            "</div>"
        ))

        # 1. Resumo de Engajamento FURIA
        self.display_furia_engagement_summary(results)

        # 2. Análise por plataforma
        for platform in ['instagram', 'twitter', 'twitch']:
            if platform in results:
                self.display_platform_analysis(platform, results[platform])

    def display_furia_engagement_summary(self, results):
        """Mostra o resumo consolidado do engajamento com a FURIA"""
        total_score = sum(data['overall_furia_score'] for data in results.values())
        avg_score = total_score / len(results) if results else 0
        overall_tier = self.analyzer.get_furia_fan_tier(avg_score)

        platform_cards = []
        for platform, data in results.items():
            platform_cards.append(
                f"<div style='display:inline-block; width:30%; margin:5px; vertical-align:top;'>"
                f"<div class='furia-card'>"
                f"<h4>{PLATFORM_ICONS[platform]} {platform.capitalize()}</h4>"
                f"<p>Score: <strong style='color:{FURIA_THEME['primary']};'>{data['overall_furia_score']}/100</strong></p>"
                f"<p>Nível: {data['furia_fan_tier']}</p>"
                f"</div>"
                f"</div>"
            )

        display(HTML(
            f"<div class='furia-card'>"
            f"<h3>📊 Resumo de Engajamento FURIA</h3>"
            f"<p style='font-size:1.2em;'>Score Médio: <strong style='color:{FURIA_THEME['primary']}; font-size:1.3em;'>{avg_score:.1f}/100</strong></p>"
            f"<p style='font-size:1.1em;'>Classificação: <strong>{overall_tier}</strong></p>"
            f"<div style='margin-top:15px;'>" + ''.join(platform_cards) + "</div>"
            f"</div>"
        ))

    def display_platform_analysis(self, platform, data):
        """Mostra análise detalhada por plataforma"""
        color = FURIA_THEME['primary']

        display(HTML(
            f"<div class='furia-card'>"
            f"<h3>{PLATFORM_ICONS[platform]} {platform.capitalize()} - @{data['username']}</h3>"
            f"<p>Score de Engajamento FURIA: <strong style='color:{color};'>{data['overall_furia_score']}/100</strong> ({data['furia_fan_tier']})</p>"
            f"</div>"
        ))

        # Métricas específicas
        metrics_html = "<div style='display: grid; grid-template-columns: repeat(auto-fill, minmax(200px, 1fr)); gap: 10px; margin: 15px 0;'>"
        for metric, value in data['metrics'].items():
            metrics_html += (
                f"<div style='background: #fff; padding: 10px; border-radius: 5px; border: 1px solid {color};'>"
                f"<strong>{metric.replace('_', ' ').title()}:</strong><br>"
                f"<span style='color:{color}; font-size:1.1em;'>{value}</span>"
                f"</div>"
            )
        metrics_html += "</div>"
        display(HTML(metrics_html))

        # Engajamento com a FURIA
        display(HTML(f"<h4><span style='color:{color};'> </span> [ Engajamento com a FURIA ]</h4>"))
        furia_engagement = data.get('furia_engagement', {})
        for category, items in furia_engagement.items():
            if items:
                if category == 'most_watched_genres':
                    display(HTML(f"<p><strong>Gêneros Mais Assistidos:</strong> {', '.join(map(str, items))}</p>"))
                elif isinstance(items, list):
                    display(HTML(f"<p><strong>{category.replace('_', ' ').title()}:</strong> {', '.join(map(str, items))}</p>"))
                else:
                    display(HTML(f"<p><strong>{category.replace('_', ' ').title()}:</strong> {str(items)}</p>"))

# %% [markdown]
### Iniciar o Dashboard de Engajamento
# %%
furia_eco_dashboard = FuriaEcoDashboard()

In [ ]:
# %% [markdown]
"""
##  Dashboard de Afinidades de Fãs de Esports
### Análise de Perfil para Engajamento e Parcerias
"""
# %%
import pandas as pd
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from ipywidgets import Layout
import random
import time
import re

AFFINITY_THEME = {
    'primary': '#FFD700',   # Dourado
    'secondary': '#000000',   # Preto
    'accent': '#FF4500'     # Laranja
}

PLATFORM_ICONS = {
    'instagram': '📷',
    'twitter': '🐦',
    'twitch': '🎮'
}

# %% [markdown]
### Módulo de Análise de Afinidades
# %%
class AffinityAnalyzer:
    def __init__(self):
        self.known_teams = ['FURIA', 'LOUD', 'paiN Gaming', 'G2 Esports', 'Team Liquid', 'Cloud9']
        self.known_players = ['s1mple', 'KSCERATO', 'yuurih', 'TenZ', 'ScreaM', 'Faker']
        self.known_sponsors = ['RedBull', 'HyperX', 'Nike', 'Logitech G', 'Secretlab', 'Adidas']
        self.known_streamers = ['gaules', 'alanzoka', 'mount', 'calango', 'casimiro']
        self.known_events = ['IEM Katowice', 'BLAST Premier', 'VCT Champions', 'CBLOL Academy']

    def analyze_instagram(self, username):
        """Análise simulada de afinidades no Instagram"""
        time.sleep(0.5)
        affinity_score = random.randint(10, 90)
        followed_teams = random.sample(self.known_teams, random.randint(0, 3))
        followed_players = random.sample(self.known_players, random.randint(0, 5))
        sponsor_mentions = random.sample(self.known_sponsors, random.randint(0, 2))
        streamer_mentions = random.sample(self.known_streamers, random.randint(0, 1))

        return {
            'username': username,
            'platform': 'instagram',
            'metrics': {
                'followers': random.randint(1000, 10000),
                'engagement_rate': f"{random.uniform(2, 7):.1f}%",
                'esports_related_posts': random.randint(5, 40)
            },
            'affinities': {
                'followed_teams': followed_teams,
                'followed_players': followed_players,
                'sponsor_mentions': sponsor_mentions,
                'streamer_mentions': streamer_mentions
            },
            'affinity_score': affinity_score
        }

    def analyze_twitter(self, username):
        """Análise simulada de afinidades no Twitter"""
        time.sleep(0.5)
        affinity_score = random.randint(15, 95)
        team_mentions = random.sample(self.known_teams, random.randint(1, 4))
        player_mentions = random.sample(self.known_players, random.randint(2, 6))
        brand_interactions = random.sample(self.known_sponsors, random.randint(0, 3))
        event_mentions = random.sample(self.known_events, random.randint(0, 2))

        return {
            'username': username,
            'platform': 'twitter',
            'metrics': {
                'followers': random.randint(500, 5000),
                'tweet_frequency': f"{random.randint(1, 8)}/dia",
                'esports_related_tweets': random.randint(10, 80)
            },
            'affinities': {
                'team_mentions': team_mentions,
                'player_mentions': player_mentions,
                'brand_interactions': brand_interactions,
                'event_mentions': event_mentions
            },
            'affinity_score': affinity_score
        }

    def analyze_twitch(self, username):
        """Análise simulada de afinidades no Twitch"""
        time.sleep(0.5)
        affinity_score = random.randint(20, 100)
        watched_streamers = random.sample(self.known_streamers, random.randint(1, 3))
        chat_interactions = random.randint(5, 50)
        event_interest = random.sample(self.known_events, random.randint(0, 1))

        return {
            'username': username,
            'platform': 'twitch',
            'metrics': {
                'followers': random.randint(100, 5000),
                'avg_viewers': random.randint(10, 300),
                'hours_watched': random.randint(20, 1500)
            },
            'affinities': {
                'watched_streamers': watched_streamers,
                'chat_interactions': chat_interactions,
                'event_interest': event_interest
            },
            'affinity_score': affinity_score
        }

    def calculate_affinity_score(self, data):
        """Calcula o Score de Afinidade (0-100)"""
        # Simplificação do cálculo de score de afinidade
        base_score = data.get('affinity_score', 50)
        data['overall_affinity_score'] = min(100, max(0, base_score))
        data['affinity_tier'] = self.get_affinity_tier(data['overall_affinity_score'])
        return data

    def get_affinity_tier(self, score):
        """Classificação do fã por nível de afinidade"""
        if score >= 80: return "Entusiasta 🔥"
        if score >= 50: return "Interessado 👍"
        if score >= 20: return "Casual 👀"
        return "Observador"

# %% [markdown]
### Dashboard de Afinidades
# %%
class AffinityDashboard:
    def __init__(self):
        self.analyzer = AffinityAnalyzer()
        self.setup_ui()

    def setup_ui(self):
        """Configura a interface do dashboard"""
        style = """
        <style>
            .affinity-header {
                background: linear-gradient(90deg, #000000 0%, #FFD700 100%);
                color: white;
                padding: 15px;
                border-radius: 8px;
                text-align: center;
                margin-bottom: 20px;
            }
            .affinity-card {
                border-left: 4px solid #FFD700;
                padding: 10px 15px;
                margin: 10px 0;
                background: #f8f8f8;
                border-radius: 0 8px 8px 0;
            }
        </style>
        """
        display(HTML(style))

        # Widgets de entrada
        self.instagram_input = widgets.Text(placeholder='instagram.com/username', description='Instagram:', layout=Layout(width='90%'))
        self.twitter_input = widgets.Text(placeholder='twitter.com/username', description='Twitter:', layout=Layout(width='90%'))
        self.twitch_input = widgets.Text(placeholder='twitch.tv/username', description='Twitch:', layout=Layout(width='90%'))
        self.analyze_btn = widgets.Button(description='Analisar Afinidades', button_style='warning', icon='search', layout=Layout(width='200px'))
        self.analyze_btn.on_click(self.analyze_profiles)
        self.output = widgets.Output()

        # Layout
        display(HTML(
            "<div class='affinity-header'>"
            "<h1>Análise de Afinidades de Fãs de Esports</h1>"
            "<p>Descubra os interesses dos fãs em times, jogadores, marcas e mais</p>"
            "</div>"
        ))

        display(widgets.VBox([
            self.instagram_input,
            self.twitter_input,
            self.twitch_input,
            self.analyze_btn,
            self.output
        ]))

    def analyze_profiles(self, b):
        """Processa as redes sociais inseridas"""
        with self.output:
            clear_output()
            display(HTML("<div style='text-align:center;'><h3>🔍 Analisando perfis...</h3></div>"))

            results = {}
            platforms = {
                'instagram': self.instagram_input.value,
                'twitter': self.twitter_input.value,
                'twitch': self.twitch_input.value
            }

            for platform, url in platforms.items():
                if url:
                    username = self.extract_username(url, platform)
                    if username:
                        data = getattr(self.analyzer, f'analyze_{platform}')(username)
                        data = self.analyzer.calculate_affinity_score(data)
                        results[platform] = data

            if results:
                self.display_results(results)
            else:
                display(HTML("<p style='color:red;text-align:center;'>Nenhuma URL válida foi fornecida</p>"))

    def extract_username(self, url, platform):
        try:
            patterns = {
                'instagram': r'(?:https?:\/\/)?(?:www\.)?instagram\.com\/([a-zA-Z0-9_.]+)/?$',
                'twitter': r'(?:https?:\/\/)?(?:www\.)?twitter\.com\/([a-zA-Z0-9_]+)/?$',
                'twitch': r'(?:https?:\/\/)?(?:www\.)?twitch\.tv\/([a-zA-Z0-9_]+)/?$'
            }
            match = re.search(patterns[platform], url.strip(), re.IGNORECASE)
            return match.group(1).lower() if match else None
        except:
            return None

    def display_results(self, results):
        """Exibe os resultados de afinidade"""
        display(HTML(
            "<div style='text-align:center; margin-bottom:20px;'>"
            f"<h2 style='color:{AFFINITY_THEME['primary']};'>Relatório de Afinidades de Esports</h2>"
            "</div>"
        ))

        # 1. Resumo de Afinidades
        self.display_affinity_summary(results)

        # 2. Análise por plataforma
        for platform in ['instagram', 'twitter', 'twitch']:
            if platform in results:
                self.display_platform_analysis(platform, results[platform])

    def display_affinity_summary(self, results):
        """Mostra o resumo consolidado das afinidades"""
        total_score = sum(data['overall_affinity_score'] for data in results.values())
        avg_score = total_score / len(results) if results else 0
        overall_tier = self.analyzer.get_affinity_tier(avg_score)

        platform_cards = []
        for platform, data in results.items():
            platform_cards.append(
                f"<div style='display:inline-block; width:30%; margin:5px; vertical-align:top;'>"
                f"<div class='affinity-card'>"
                f"<h4>{PLATFORM_ICONS[platform]} {platform.capitalize()}</h4>"
                f"<p>Score: <strong style='color:{AFFINITY_THEME['primary']};'>{data['overall_affinity_score']}/100</strong></p>"
                f"<p>Nível: {data['affinity_tier']}</p>"
                f"</div>"
                f"</div>"
            )

        display(HTML(
            f"<div class='affinity-card'>"
            f"<h3>📊 Resumo de Afinidades</h3>"
            f"<p style='font-size:1.2em;'>Score Médio: <strong style='color:{AFFINITY_THEME['primary']}; font-size:1.3em;'>{avg_score:.1f}/100</strong></p>"
            f"<p style='font-size:1.1em;'>Classificação: <strong>{overall_tier}</strong></p>"
            f"<div style='margin-top:15px;'>" + ''.join(platform_cards) + "</div>"
            f"</div>"
        ))

    def display_platform_analysis(self, platform, data):
        """Mostra análise detalhada por plataforma"""
        color = AFFINITY_THEME['primary']

        display(HTML(
            f"<div class='affinity-card'>"
            f"<h3>{PLATFORM_ICONS[platform]} {platform.capitalize()} - @{data['username']}</h3>"
            f"<p>Score de Afinidade: <strong style='color:{color};'>{data['overall_affinity_score']}/100</strong> ({data['affinity_tier']})</p>"
            f"</div>"
        ))

        # Métricas específicas
        metrics_html = "<div style='display: grid; grid-template-columns: repeat(auto-fill, minmax(200px, 1fr)); gap: 10px; margin: 15px 0;'>"
        for metric, value in data['metrics'].items():
            metrics_html += (
                f"<div style='background: #fff; padding: 10px; border-radius: 5px; border: 1px solid {color};'>"
                f"<strong>{metric.replace('_', ' ').title()}:</strong><br>"
                f"<span style='color:{color}; font-size:1.1em;'>{value}</span>"
                f"</div>"
            )
        metrics_html += "</div>"
        display(HTML(metrics_html))

        # Afinidades
        display(HTML(f"<h4><span style='color:{color};'> </span> [ Afinidades Encontradas ]</h4>"))
        affinities = data.get('affinities', {})
        for category, items in affinities.items():
            if items and category != 'player_streams_watched':
                if isinstance(items, list):
                    display(HTML(f"<p><strong>{category.replace('_', ' ').title()}:</strong> {', '.join(map(str, items))}</p>"))
                else:
                    display(HTML(f"<p><strong>{category.replace('_', ' ').title()}:</strong> {str(items)}</p>"))

# %% [markdown]
### Iniciar o Dashboard de Afinidades
# %%
affinity_dashboard = AffinityDashboard()

In [ ]:
# %% [markdown]
# ## Sistema de Análise de Perfis

# %%
import pandas as pd
from transformers import pipeline
import numpy as np
import re
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import Layout
import torch

# Configuração do classificador com tratamento de erros
try:
    classifier = pipeline("zero-shot-classification",
                          model="facebook/bart-large-mnli",
                          device=0 if torch.cuda.is_available() else -1)
except Exception as e:
    print(f"Erro ao inicializar o classificador com GPU: {e}")
    classifier = pipeline("zero-shot-classification",
                          model="facebook/bart-large-mnli", device=-1)

# %%
class ProfileAnalyzer:

    def __init__(self):
        self.platform_apis = {
            'twitter': self._get_twitter_content,
            'twitch': self._get_twitch_content,
            'instagram': self._get_instagram_content
        }
        self.min_followers = 1000  # Valor mínimo para evitar divisão por zero
        self.keywords_furia = ["FURIA", "furia", "Furia", "FÚRIA", "Pantera", "KSCERATO", "yuurih", "skullz", "chelobit", "VemPraFuria"]
        self.keywords_esports = ["esports", "e-sports", "gaming", "gamer", "pro player", "campeonato", "liga", "major", "mundial"]
        self.keywords_csgo = ["CS:GO", "csgo", "Counter-Strike", "мираж", "inferno", "dust2"]
        self.keywords_valorant = ["Valorant", "val", "Riot", "fps", "agents", "мираж", "icebox", "ascent"]

    def _get_platform(self, url):
        """Identifica a plataforma pela URL"""
        url = url.lower()
        if 'twitter' in url:
            return 'twitter'
        elif 'twitch' in url:
            return 'twitch'
        elif 'instagram' in url:
            return 'instagram'
        return 'other'

    def _get_twitter_content(self, url):
        """Conteúdo mock para Twitter"""
        mock_data = {
            "https://twitter.com/furiagg": {
                "content": "Estamos na GRANDE FINAL do campeonato de #CSGO! A #FURIA vai com tudo! #VemPraFuria",
                "followers": 550000,
                "engagement": 0.06
            },
            "https://twitter.com/csgobrasil": {
                "content": "Notícias de #CSGO: A FURIA venceu a semifinal do major! #esports",
                "followers": 220000,
                "engagement": 0.04
            },
            "https://twitter.com/gaules": {
                "content": "AO VIVO transmitindo a FURIA no major de CS:GO! Cheguem mais, família! #Gaules #CSGO #FURIA",
                "followers": 1800000,
                "engagement": 0.09
            },
            "https://twitter.com/skullzcsgo": {
                "content": "Preparado para mais um dia de treino com a #FURIA! #CSGOpro",
                "followers": 150000,
                "engagement": 0.07
            }
        }
        return mock_data.get(url, {"content": f"Tweet sobre e-sports ({url})", "followers": 12000, "engagement": 0.015})

    def _get_twitch_content(self, url):
        """Conteúdo mock para Twitch"""
        mock_data = {
            "https://twitch.tv/furiagg": {
                "content": "🔴 AO VIVO: FURIA vs LOUD no Valorant! Vem com a gente! #Valorant #FURIA",
                "viewers": 18000,
                "followers": 350000
            },
            "https://twitch.tv/valorant_br": {
                "content": "Acompanhe o campeonato brasileiro de #Valorant com a participação da FURIA!",
                "viewers": 10000,
                "followers": 150000
            },
            "https://twitch.tv/gaules": {
                "content": "Assistindo e comentando os jogos da FURIA no CS:GO major! #CSGO #FURIA #Gaules",
                "viewers": 60000,
                "followers": 2500000
            },
            "https://twitch.tv/yuurih": {
                "content": "Jogando umas ranqueadas de #CSGO! Passa aí pra trocar uma ideia!",
                "viewers": 8000,
                "followers": 200000
            }
        }
        return mock_data.get(url, {"content": f"Stream de e-sports ({url})", "viewers": 1500, "followers": 7000})

    def _get_instagram_content(self, url):
        """Conteúdo mock para Instagram"""
        mock_data = {
            "https://instagram.com/fansfuria": {
                "content": "Foto incrível da line-up de #CSGO da #FURIA no último campeonato!",
                "likes": 7000,
                "followers": 120000
            },
            "https://instagram.com/csgo_world": {
                "content": "Os melhores momentos da #FURIA no major de #CSGO! #esports",
                "likes": 4500,
                "followers": 95000
            },
            "https://instagram.com/furiagg": {
                "content": "A energia da torcida #FURIA é INCRÍVEL! #Valorant",
                "likes": 12000,
                "followers": 600000
            },
            "https://instagram.com/kscerato": {
                "content": "Focado nos treinos de #CSGO com a #FURIA!",
                "likes": 9000,
                "followers": 250000
            }
        }
        return mock_data.get(url, {"content": f"Post de e-sports ({url})", "likes": 150, "followers": 1500})

    def _score_keywords(self, text, keywords, weight):
        """Calcula um score baseado na presença de keywords."""
        if not text or not isinstance(text, str):
            return 0.0
        count = sum(1 for keyword in keywords if re.search(r'\b' + re.escape(keyword) + r'\b', text, re.IGNORECASE))
        return count * weight

    def analyze_relevance(self, text):
        """Calcula a relevância para FURIA usando zero-shot e keywords."""
        if not text or not isinstance(text, str):
            return 0.0

        try:
            result = classifier(text, ["FURIA Esports", "CS:GO", "Valorant", "e-sports", "conteúdo genérico"])
            furia_score_zero_shot = result['scores'][result['labels'].index("FURIA Esports")] if "FURIA Esports" in result['labels'] else 0.0

            keyword_score_furia = self._score_keywords(text, self.keywords_furia, 0.4)
            keyword_score_esports = self._score_keywords(text, self.keywords_esports, 0.2)
            keyword_score_csgo = self._score_keywords(text, self.keywords_csgo, 0.1)
            keyword_score_valorant = self._score_keywords(text, self.keywords_valorant, 0.1)

            # Combina os scores, dando mais peso ao zero-shot
            combined_score = (furia_score_zero_shot * 0.5 +
                              keyword_score_furia +
                              keyword_score_esports +
                              keyword_score_csgo +
                              keyword_score_valorant)

            return min(max(combined_score, 0.0), 1.0)  # Garante entre 0 e 1
        except Exception as e:
            print(f"Erro na análise de relevância: {e}")
            return 0.0

    def analyze_profile(self, url):

        try:
            platform = self._get_platform(url)
            content_data = self.platform_apis.get(platform, lambda x: {"content": ""})(url)

            # Garante valores padrão
            content_data.setdefault('content', '')
            content_data.setdefault('followers', self.min_followers)
            content_data.setdefault('engagement', 0.0)
            content_data.setdefault('viewers', 0)
            content_data.setdefault('likes', 0)

            # Calcula a relevância
            relevance_score = self.analyze_relevance(content_data['content'])

            return {
                'profile_url': url,
                'platform': platform,
                'relevance_score': relevance_score,
                'followers': content_data['followers'],
                'engagement': content_data['engagement'],
                'viewers': content_data.get('viewers', 0),
                'likes': content_data.get('likes', 0)
            }

        except Exception as e:
            print(f"Erro ao analisar {url}: {e}")
            return {
                'profile_url': url,
                'platform': self._get_platform(url),
                'relevance_score': 0.0,
                'followers': self.min_followers,
                'engagement': 0.0,
                'viewers': 0,
                'likes': 0
            }

# %%
# Processamento principal completo
analyzer = ProfileAnalyzer()

profile_urls = [
    "https://twitter.com/furiagg",
    "https://twitter.com/csgobrasil",
    "https://twitch.tv/furiagg",
    "https://twitch.tv/valorant_br",
    "https://instagram.com/fansfuria",
    "https://twitch.tv/gaules",
    "https://instagram.com/csgo_world",
    "https://twitter.com/arTcsgo",
    "https://twitch.tv/yuurih",
    "https://instagram.com/kscerato",
    "https://www.youtube.com/somegamingchannel" # Exemplo de outra plataforma
]

# Coleta e análise dos dados
results = []
for url in profile_urls:
    results.append(analyzer.analyze_profile(url))

# Cria DataFrame
profiles_df = pd.DataFrame(results)

# Cálculo do composite_score com pesos ajustáveis
def calculate_composite_score(row):
    normalized_followers = row['followers'] / max(profiles_df['followers'].max(), 1)
    return (row['relevance_score'] * 0.5 +
            normalized_followers * 0.3 +
            row['engagement'] * 0.1 +
            (row['viewers'] / max(profiles_df['viewers'].max(), 1) * 0.05 if 'viewers' in row and profiles_df['viewers'].max() > 0 else 0) +
            (row['likes'] / max(profiles_df['likes'].max(), 1) * 0.05 if 'likes' in row and profiles_df['likes'].max() > 0 else 0))

profiles_df['composite_score'] = profiles_df.apply(calculate_composite_score, axis=1)

# Classificação de relevância com bins dinâmicos e tratamento de poucos perfis
if len(profiles_df) > 2:
    bins = pd.qcut(profiles_df['composite_score'], q=3, labels=['baixa', 'média', 'alta'])
else:
    bins = pd.cut(profiles_df['composite_score'], bins=3, labels=['baixa', 'média', 'alta'], include_lowest=True)
profiles_df['relevance_tier'] = bins.astype(str) # Garante que é string para evitar problemas em visualização

# Prepara a saída final
output_columns = ['profile_url', 'platform', 'relevance_tier', 'composite_score', 'followers', 'engagement', 'viewers', 'likes']
final_df = profiles_df[output_columns].sort_values('composite_score', ascending=False).fillna(0)

# Formatação final
final_df['composite_score'] = final_df['composite_score'].round(6)
final_df['engagement'] = final_df['engagement'].round(6)

print("\nResultado Final da Análise:\n")
print(final_df.to_string(index=False))



In [ ]:
# %% [markdown]
"""
## 5. Exportação dos Dados
"""
# %%
# Widget para exportação
btn_exportar = widgets.Button(description="Exportar Dados para CSV")

def exportar_dados(b):
    if df_fans.empty:
        print("Nenhum dado para exportar.")
        return

    df_fans.to_csv('dados_fans_esports.csv', index=False)
    print("Dados exportados com sucesso para 'dados_fans_esports.csv'")

btn_exportar.on_click(exportar_dados)

display(btn_exportar)